# Reports

In [1]:
import os
import re
from datetime import datetime
current_datetime = datetime.now().strftime('%Y%m%d.%H%M')

import numpy as np
import pandas as pd

# for the notebook rendering 
from IPython.display import display, HTML, FileLink, Markdown
from IPython.display import FileLink as FL
from IPython.display import Markdown as MD


DIR = os.getcwd()
print(f'{DIR=}')

RDIR = os.path.join(DIR,'reports')
print(f'{RDIR=}')

DIR='c:\\Users\\JGarza\\GitHub\\wifi_monitor'
RDIR='c:\\Users\\JGarza\\GitHub\\wifi_monitor\\reports'


In [2]:
def break_down_buckets(idf,column,buckets,message='',nan_value=-1):
    """
    breaks a column down into buckets/bins
    """
    idf = idf.fillna(nan_value)

    print('',message,'\ncolumn: ',column, '\nbuckets: ', buckets)
    
    idf = pd.DataFrame(idf[column])
    idf['bucket'] = pd.cut(idf[column], bins=buckets)
    idf = idf.groupby(by='bucket').count()
    idf['percent'] = (idf[column]/idf[column].sum())*100
    idf['percent'] = idf['percent'].round(2)
    display(idf)

    return idf

#test
# break_down_buckets(df,'AGE_YRS',[-1,0,15,25,35,45,55,65,75,85,500])

In [3]:
def bar(num,denom=100.0,length=25,fillchar='#',emptychar='_'):
    fillnum = ((int)( (num/denom) * length))
    return '[' + ( fillnum * fillchar ).ljust(length,emptychar)  + ']' # + f" {(num/denom)*100.0:.2f}%     " 

In [4]:
# YYYYMMDDHHmm
DateFilter = None
# DateFilter = 202501010000

In [5]:
data = {}

for i in os.listdir(RDIR):
    # print(i)
    try:

        text = ''
        with open(os.path.join(RDIR, i), 'r') as f:
            text = f.read()
        # print(text)

        try:
            if DateFilter != None:
                datetime = re.findall('Datetime_alt: .*\n',text)[0]
                datetime = datetime.replace('Datetime_alt: ','').replace('\\n','')
                datetime = int(datetime)
                
                
                if datetime < DateFilter:
                    #move on to next
                    print('exclude',datetime,DateFilter)
                    continue
                else:
                    print('include',datetime,DateFilter)
                
        except:
            pass


        wifi = re.findall('^SSID:.*\n',text)[0]
        wifi = wifi.replace('\n', '').replace('SSID: ','')

        pings = re.findall('.*time=.*\n',text)
        pings = [ re.sub('.*time=','',p) for p in pings]
        pings = [ re.sub('ms.*\n','',p) for p in pings]
        pings = [ int(p) for p in pings]
        # print(*pings,sep='\n')

        timeouts = re.findall('Request timed out.\n',text)

        trycount = re.findall('(Request|Reply).*\n',text)

        if wifi in data.keys():
            pass
        else:
            data[wifi] = {}
            data[wifi]['pings'] = []
            data[wifi]['timeouts'] = []
            data[wifi]['trycount'] = []
        
        data[wifi]['pings'].extend(pings)
        data[wifi]['timeouts'].extend([len(timeouts)])
        data[wifi]['trycount'].extend([len(trycount)])

    except Exception as e:
        pass
        # print(i,e)

# for k in data.keys():
#     print('-'*20)
#     print(k)
#     df_pings = pd.DataFrame(data[k]['pings'])
#     print(df_pings[0].describe())
#     print()
#     print(f'Request timed out:')
#     print(f'{ sum(data[k]['timeouts']) } out of { sum(data[k]['trycount']) }')
#     print(f'{ sum(data[k]['timeouts']) / sum(data[k]['trycount']) }')

file = os.path.join(DIR,'docs','index.html')
with open(file,'w',encoding='utf8') as f:

    f.write(f'<h>{current_datetime}</h>')

    for k in data.keys():
        
        f.write('<p>' + '-'*20 + '</p>')

        f.write(f'<p>{k}</p>')
        df_pings = pd.DataFrame(data[k]['pings'])

        csvlines = df_pings.describe().to_csv()
        f.write(csvlines[4::].replace('\n','<br \>').replace(',','\t'))

        # print(df_pings)

        bdb = break_down_buckets(df_pings,0,[0,5,10,15,20,30,40,50,100,250,500])
        bdb['bar'] = bdb.percent.apply(bar)
        # display(bdb)

        f.write('<br \>')
        f.write(bdb.to_html())
        f.write('<br \>')

        f.write(f'Request timed out (failed pings):<br \>')
        f.write(f'{ sum(data[k]['timeouts']) } out of { sum(data[k]['trycount']) }<br \>')
        f.write(f'{ sum(data[k]['timeouts']) / sum(data[k]['trycount']) }')




  
column:  0 
buckets:  [0, 5, 10, 15, 20, 30, 40, 50, 100, 250, 500]


<>:82: SyntaxWarning: invalid escape sequence '\>'
<>:90: SyntaxWarning: invalid escape sequence '\>'
<>:92: SyntaxWarning: invalid escape sequence '\>'
<>:94: SyntaxWarning: invalid escape sequence '\>'
<>:95: SyntaxWarning: invalid escape sequence '\>'
<>:82: SyntaxWarning: invalid escape sequence '\>'
<>:90: SyntaxWarning: invalid escape sequence '\>'
<>:92: SyntaxWarning: invalid escape sequence '\>'
<>:94: SyntaxWarning: invalid escape sequence '\>'
<>:95: SyntaxWarning: invalid escape sequence '\>'
C:\Users\JGarza\AppData\Local\Temp\ipykernel_3952\1602092522.py:82: SyntaxWarning: invalid escape sequence '\>'
  f.write(csvlines[4::].replace('\n','<br \>').replace(',','\t'))
C:\Users\JGarza\AppData\Local\Temp\ipykernel_3952\1602092522.py:90: SyntaxWarning: invalid escape sequence '\>'
  f.write('<br \>')
C:\Users\JGarza\AppData\Local\Temp\ipykernel_3952\1602092522.py:92: SyntaxWarning: invalid escape sequence '\>'
  f.write('<br \>')
C:\Users\JGarza\AppData\Local\Temp\ipykernel_395

,0,percent
bucket,,
"(0, 5]",0,0.00
"(5, 10]",220,20.09
"(10, 15]",660,60.27
"(15, 20]",81,7.40
"(20, 30]",51,4.66
"(30, 40]",22,2.01
"(40, 50]",18,1.64
"(50, 100]",20,1.83
"(100, 250]",16,1.46


  
column:  0 
buckets:  [0, 5, 10, 15, 20, 30, 40, 50, 100, 250, 500]


C:\Users\JGarza\AppData\Local\Temp\ipykernel_3952\3656564501.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  idf = idf.groupby(by='bucket').count()


,0,percent
bucket,,
"(0, 5]",0,0.00
"(5, 10]",539,49.04
"(10, 15]",463,42.13
"(15, 20]",44,4.00
"(20, 30]",38,3.46
"(30, 40]",8,0.73
"(40, 50]",4,0.36
"(50, 100]",2,0.18
"(100, 250]",1,0.09


  
column:  0 
buckets:  [0, 5, 10, 15, 20, 30, 40, 50, 100, 250, 500]


C:\Users\JGarza\AppData\Local\Temp\ipykernel_3952\3656564501.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  idf = idf.groupby(by='bucket').count()


,0,percent
bucket,,
"(0, 5]",0,0.00
"(5, 10]",37,3.37
"(10, 15]",667,60.75
"(15, 20]",240,21.86
"(20, 30]",82,7.47
"(30, 40]",31,2.82
"(40, 50]",18,1.64
"(50, 100]",15,1.37
"(100, 250]",5,0.46
